In [ ]:
# !pip install konlpy


In [ ]:
from konlpy.tag import Hannanum, Kkma, Okt, Komoran

komo = Komoran()
hannanum = Hannanum()
kkma = Kkma()
okt = Okt()

In [ ]:
import numpy as np
import re
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras



In [ ]:
# 데이터 불러오기
news = pd.read_csv("/content/drive/My Drive//23035news_content_headline.csv")
# news = news.append(dff, ignore_index=True)
news = news.loc[:, ['headline', 'content']]


# news.to_csv('23035news_content_headline.csv',encoding='utf-8-sig')

news = news.dropna(axis=0)
# 불용어 제거
# 영어 대 소문자 제거
sw = '[a-zA-Z]'
news['content'] = news['content'].str.replace(sw, ' ')
news['headline'] = news['headline'].str.replace(sw, ' ')

# 한글과 숫자를 제외한 특수문자 제거
news['content'] = news['content'].str.replace('[^\w]', ' ' )
news['headline'] = news['headline'].str.replace('[^\w]', ' ')

# 기사 스탑워드 제거
news['content'] = news['content'].str.replace('기자의 다른기사 더보기', ' ')



# 중복되는 헤드라인 행 제거
news = news.drop_duplicates(['headline'])


print(news)
print(news.content[1011:1020])
print(news.headline[1011:1020])

In [ ]:
# # 미리 토큰화된 것 부르기

# encoder_input = []
# with open('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/18000encoder_input(kkma).txt', 'r') as f:
#     for line in f.readlines():
#         encoder_input.append(line.split())
# decoder_input = []
# with open('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/18000decoder_input(kkma).txt', 'r') as f:
#     for line in f.readlines():
#         decoder_input.append(line.split())
# decoder_output = []
# with open('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/18000decoder_output(kkma).txt', 'r') as f:
#     for line in f.readlines():
#         decoder_output.append(line.split())

In [ ]:
# 토큰화
# 형태소추출

encoder_input, decoder_input, decoder_output = [], [], []

i = 0

for stc in news['content']:
    morphs = kkma.morphs(stc)
    if i% 100 == 0:
        print(i)
    i+=1
    encoder_input.append(morphs)
i = 0
for stc in news['headline']:
    morphs = kkma.morphs(stc)
    if i% 100 == 0:
        print(i)
    i+=1
    decoder_input.append(["<start>"]+ morphs)
i = 0
for stc in news['headline']:
    morphs = kkma.morphs(stc)
    if i% 100 == 0:
        print(i)
    i+=1
    decoder_output.append(morphs + ["<end>"])

In [ ]:
# 토큰화 된 것 부르기
with open('encoder_input.txt(kkma)', 'w') as f:
    for line in encoder_input:
        f.writelines(' '.join(line))
        f.write('\n')

with open('decoder_input.txt(kkma)', 'w') as f:
    for line in decoder_input:
        f.writelines(' '.join(line))
        f.write('\n')
        
with open('decoder_output.txt(kkma)', 'w') as f:
    for line in decoder_output:
        f.writelines(' '.join(line))
        f.write('\n')

In [ ]:
# with open('encoder_input.txt(komoran)', 'w') as f:
#     for line in encoder_input:
#         f.writelines(' '.join(line))
#         f.write('\n')
# with open('decoder_input.txt(komoran)', 'w') as f:
#     for line in decoder_input:
#         f.writelines(' '.join(line))
#         f.write('\n')
# with open('decoder_output.txt(komoran)', 'w') as f:
#     for line in decoder_output:
#         f.writelines(' '.join(line))
#         f.write('\n')

In [ ]:
print(encoder_input[:10])
print(decoder_input[:10])
print(decoder_output[:10])

In [ ]:
import matplotlib.pyplot as plt


# 길이 분포 출력
text_len = [len(s) for s in encoder_input]
summary_len = [len(s) for s in decoder_input]

print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
print('요약의 최소 길이 : {}'.format(np.min(summary_len)))
print('요약의 최대 길이 : {}'.format(np.max(summary_len)))
print('요약의 평균 길이 : {}'.format(np.mean(summary_len)))

plt.subplot(1,2,1)
plt.boxplot(summary_len)
plt.title('contents')
plt.subplot(1,2,2)
plt.boxplot(text_len)
plt.title('title')
plt.tight_layout()
plt.show()

plt.title('contents')
plt.hist(summary_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

plt.title('title')
plt.hist(text_len, bins=40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 정수 인코딩
tokenizer_con = Tokenizer()
tokenizer_con.fit_on_texts(encoder_input)
encoder_input_s = tokenizer_con.texts_to_sequences(encoder_input)

tokenizer_head = Tokenizer()
tokenizer_head.fit_on_texts(decoder_input)
tokenizer_head.fit_on_texts(decoder_output)
decoder_input_s = tokenizer_head.texts_to_sequences(decoder_input)
decoder_output_s = tokenizer_head.texts_to_sequences(decoder_output)

# 패딩
encoder_input_pad = pad_sequences(encoder_input_s, padding="post", maxlen=1000)
decoder_input_pad = pad_sequences(decoder_input_s, padding="post", maxlen=25)
decoder_output_pad = pad_sequences(decoder_output_s, padding="post",maxlen=25)

print(encoder_input_pad.shape)
print(decoder_input_pad.shape)

head_to_index = tokenizer_head.word_index
index_to_head = tokenizer_head.index_word


# suffle
mb_size = 256
input_rows = len(encoder_input)

shuffle_map = np.arange(input_rows)#################################
np.random.shuffle(shuffle_map)
step_count = int(input_rows * 0.8) // mb_size
test_begin_idx = step_count * mb_size




# train, test 데이터 분리

encoder_input_train = encoder_input_pad[shuffle_map[:test_begin_idx]]
decoder_input_train = decoder_input_pad[shuffle_map[:test_begin_idx]]
decoder_output_train = decoder_output_pad[shuffle_map[:test_begin_idx]]

encoder_input_test = encoder_input_pad[shuffle_map[test_begin_idx:]]
decoder_input_test = decoder_input_pad[shuffle_map[test_begin_idx:]]
decoder_output_test = decoder_output_pad[shuffle_map[test_begin_idx:]]

In [ ]:
type(encoder_input_pad)

numpy.ndarray

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
embedding_dim = 128
hidden_size = 64

# 인코더 모델 쌓기
encoder_inputs = Input(shape=(1000,))
# 인코더의 임베딩 층
encoder_embed = Embedding(len(tokenizer_con.word_index)+1, embedding_dim)(encoder_inputs)
# 패딩(0)에 해당하는 임베딩 벡터는 제외 -- masking
encoder_mask = Masking(mask_value=0)(encoder_embed)

# encoder_outputs, h_state, c_state = LSTM(hidden_size, return_state=True, return_sequences=True)(encoder_mask)
# 인코더의 LSTM 1
encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True )
encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_mask) # h1,c1 은 사용 안함

# 인코더의 LSTM 2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) # h2,c2 은 사용 안함

# 인코더의 LSTM 3
# return sequences = True 를 통해서 어텐션을 구할 때 필요한 전체 시점의 히든 상태값을 리턴하도록!
# 드롭아웃 사용 -- 과적합 방지
encoder_lstm3 = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.4, recurrent_dropout=0.2) 
encoder_outputs, h_state, c_state = encoder_lstm3(encoder_output2)




# 디코더 모델 쌓기
decoder_inputs = Input(shape=(25,))
decoder_embed = Embedding(len(tokenizer_head.word_index)+1, embedding_dim)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])




from attention import AttentionLayer

# 어텐션 레이어 객체 생성
attn_layer = AttentionLayer()
# attn_out는 어텐션 밸류 (가중치가 보정된 인코더의 은닉 상태값의 합), attn_states는 가중치 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
# 디코더의 히든상태랑 어텐션 밸류를 결합해서 새로운 출력 벡터 구함
decoder_concat_input = Concatenate()([decoder_outputs, attn_out])

decoder_dense = Dense(len(tokenizer_head.word_index)+1, activation='softmax')
decoder_softmax_outputs = decoder_dense(decoder_concat_input)



In [ ]:
# # LSTM 한층만 사용 -- 속도..

# from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# embedding_dim = 128
# hidden_size = 64

# # 인코더 모델 쌓기
# encoder_inputs = Input(shape=(1000,))
# # 인코더의 임베딩 층
# encoder_embed = Embedding(len(tokenizer_con.word_index)+1, embedding_dim)(encoder_inputs)
# # 패딩(0)에 해당하는 임베딩 벡터는 제외 -- masking
# encoder_mask = Masking(mask_value=0)(encoder_embed)
# encoder_outputs, h_state, c_state = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)(encoder_mask)

# # 디코더 모델 쌓기
# decoder_inputs = Input(shape=(20,))
# decoder_embed = Embedding(len(tokenizer_head.word_index)+1, embedding_dim)(decoder_inputs)
# decoder_mask = Masking(mask_value=0)(decoder_embed)
# decoder_lstm = LSTM(hidden_size, return_sequences=True, return_state=True,dropout=0.2, recurrent_dropout=0.1)
# decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])




# from attention import AttentionLayer

# # 어텐션 레이어 객체 생성
# attn_layer = AttentionLayer()
# # attn_out는 어텐션 밸류 (가중치가 보정된 인코더의 은닉 상태값의 합), attn_states는 가중치 
# attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
# # 디코더의 히든상태랑 어텐션 밸류를 결합해서 새로운 출력 벡터 구함
# decoder_concat_input = Concatenate()([decoder_outputs, attn_out])

# decoder_dense = Dense(len(tokenizer_head.word_index)+1, activation='softmax')
# decoder_softmax_outputs = decoder_dense(decoder_concat_input)



In [ ]:
cp = ModelCheckpoint(filepath ='/content/drive/My Drive/best_kkma_model.h5' ,monitor='val_loss', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=30)

# 모델 훈련
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_output_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_output_test), batch_size = mb_size, epochs = 200, callbacks=[cp,es])



In [ ]:
model.summary()


In [ ]:
from keras.models import load_model
# 모델 저장
model.save('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/kkma_model.h5')

In [ ]:
# 어텐션 계산을 위해 출력으로 encoder_outputs 까지!
encoder_model = Model(encoder_inputs, [encoder_outputs, h_state, c_state])


encoder_h_state = Input(shape=(hidden_size,))
encoder_c_state = Input(shape=(hidden_size,))

pd_decoder_outputs, pd_h_state, pd_c_state = decoder_lstm(decoder_mask, initial_state=[encoder_h_state, encoder_c_state])

# 어텐션
# 1000 시점 (단어, 패딩) 의 수, hidden_size는 히든 스테이트의 차원
pd_encoder_outputs = Input(shape=(1000, hidden_size))
pd_attn_out, pd_attn_states = attn_layer([pd_encoder_outputs, pd_decoder_outputs])
pd_decoder_concat = Concatenate()([pd_decoder_outputs, pd_attn_out])

pd_decoder_softmax_outputs = decoder_dense(pd_decoder_concat)

# 어텐션은 디코더 모델 안에서 사용하는거기 때문에, 인풋으로 encoder outputs 까지 넣어준다!

decoder_model = Model([decoder_inputs, pd_encoder_outputs, encoder_h_state, encoder_c_state], [pd_decoder_softmax_outputs, pd_h_state, pd_c_state])

In [ ]:
# 인코더 디코더 모델 저장
encoder_model.save('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/kkma_encoder_model.h5')
decoder_model.save('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/kkma_decoder_model.h5')


In [ ]:
import matplotlib.pyplot as plt


plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(history.history['acc'], label='train_acc')
plt.plot(history.history['val_acc'], label='test_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



# 모델불러오기

In [ ]:
encoder_model = keras.models.load_model('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/kkma_encoder_model.h5')

In [ ]:
from attention import AttentionLayer
decoder_model = keras.models.load_model('/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/kkma_decoder_model.h5', custom_objects={'AttentionLayer': AttentionLayer})

# 내용입력부분

In [ ]:
# 전체의 10퍼센트 정도의 데이터를 예측
# suffle
mb_size = 256
input_rows = len(encoder_input)

shuff = np.arange(input_rows)#################################
np.random.shuffle(shuff)
step_count = int(input_rows * 0.9) // mb_size
test_begin_idx = step_count * mb_size

# array로 type변경
cont = news.content[:16500]
cont = cont.to_numpy()



predicted_title = []

for inp in cont[shuff[test_begin_idx:]]:
    input_stc = inp
    token_stc = kkma.morphs(input_stc)
    encode_stc = tokenizer_con.texts_to_sequences([token_stc])
    pad_stc = pad_sequences(encode_stc, maxlen=1000, padding="post")

    # 출력이 3가지 (전체 시점의 히든 상태값, 마지막 시점의 히든/셀 상태 값) 가 나온다
    en_out, en_hidden, en_cell = encoder_model.predict(pad_stc)

    predicted_seq = np.zeros((1,1))
    predicted_seq[0, 0] = head_to_index['<start>']

    decoded_stc = []

    while True:
        # 여기서 인풋으로 en_out 도 같이 넣어준다!
        output_words, h, c = decoder_model.predict([predicted_seq, en_out, en_hidden, en_cell])
        # print(index_to_head)
        
        predicted_word = index_to_head[np.argmax(output_words[0,0])]
        # print(predicted_word)
        
        if predicted_word == '<end>':
            break
        
        decoded_stc.append(predicted_word)

        predicted_seq = np.zeros((1,1))
        predicted_seq[0, 0] = np.argmax(output_words[0, 0])

        en_hidden = h
        en_cell = c

    result = ' '.join(decoded_stc)
    predicted_title.append(result)
    print(result)




In [ ]:
len(predicted_title)

5726

In [ ]:
label_title = []
head = news.headline[:16500]
head = head.to_numpy()
i = 0
for inp in head[shuff[test_begin_idx:]]:
    label_title.append(inp)
    i += 1
    if i == 5726:
        break

contents = []
i = 0
for inp in cont[shuff[test_begin_idx:]]:
    contents.append(inp)
    i += 1
    if i == 5726:
        break


print(len(label_title))
print(len(contents))





5726
5726


In [ ]:
title = {'labeled_title' : label_title, 'predicted_title': predicted_title, 'content' : contents}
title_df = pd.DataFrame(title)
title_df
title_df.to_csv("/content/drive/My Drive/인공지능 학습관련/자연어 프로젝트/labeled_predicted_title_kkma.csv", mode='w', encoding='utf-8-sig')
